# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/weather_queries.csv")


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create a list containing coordinates
coordinates = list(zip(df["Lat"],df["Lon"]))
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [4]:
# Assign the marker layer to a variable
heatmap = gmaps.heatmap_layer(coordinates, weights=df["Humidity"])
# Add the layer to the map
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
conditions = (df["Max Temp"]>70) & (df["Max Temp"]<80) & (df["Cloudiness"]==0)
narrowed_city_df = df[conditions]
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
4,Saint-Philippe,0,RE,1584651945,64,-21.36,55.77,75.20,9.17
156,Marsh Harbour,0,BS,1584651913,75,26.54,-77.06,74.23,16.58
166,Guhāgar,0,IN,1584651982,73,17.48,73.19,79.99,12.01
172,Cidreira,0,BR,1584651983,75,-30.18,-50.21,76.55,13.96
204,Yulara,0,AU,1584651991,29,-25.24,130.99,75.20,9.17
220,Taoudenni,0,ML,1584651994,14,22.68,-3.98,72.25,10.78
224,Richards Bay,0,ZA,1584651995,86,-28.78,32.04,75.13,8.59
252,Antsohihy,0,MG,1584652002,95,-14.88,47.99,74.05,0.13
289,Saint-Joseph,0,RE,1584652011,64,-21.37,55.62,75.20,9.17
293,Venado Tuerto,0,AR,1584652012,54,-33.75,-61.97,78.19,3.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_name=[]
lat=[]
lon=[]
hotel_city_name=[]
hotel_country=[]

for (city_lat, city_lon) in zip(narrowed_city_df["Lat"],narrowed_city_df["Lon"]):


    # geocoordinates
    target_coordinates = f"{city_lat},{city_lon}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
  #      "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()

 #   print(response.url)
    try:
        lat.append(places_data["results"][0]["geometry"]["location"]["lat"])

        lon.append(places_data["results"][0]["geometry"]["location"]["lng"])

        hotel_name.append(places_data["results"][0]["name"])
        
        city_record=narrowed_city_df.loc[(narrowed_city_df["Lat"]==city_lat) &(narrowed_city_df["Lon"]==city_lon)]
        hotel_city_name=hotel_city_name+ city_record["City"].tolist()
        hotel_country= hotel_country+ city_record["Country"].tolist()
    except:
        #print("No results, passing..")
        pass


hotel_df = pd.DataFrame({
    "Hotel Name":hotel_name,
    "Lat":lat,
    "Lon":lon,
    "City": hotel_city_name,
    "Country": hotel_country
})



In [7]:

hotel_df


,Hotel Name,Lat,Lon,City,Country
0,Les Embruns Du Baril,-21.369201,55.727949,Saint-Philippe,RE
1,Abaco Beach Resort,26.546252,-77.046387,Marsh Harbour,BS
2,Nilambari Beach Resort,17.486467,73.188427,Guhāgar,IN
3,Hotel Castelo,-30.177881,-50.203501,Cidreira,BR
4,Desert Gardens Hotel - Ayers Rock Resort,-25.243725,130.984807,Yulara,AU
5,BON Hotel Waterfront Richards Bay,-28.793297,32.080793,Richards Bay,ZA
6,Hotel Belle Vue,-14.898462,47.981034,Antsohihy,MG
7,Manapescale (Chambre d'hôtes),-21.375881,55.598440,Saint-Joseph,RE
8,Pedrino Hotel,-33.719839,-61.991829,Venado Tuerto,AR
9,3G hotel pasni,25.263623,63.470265,Pasni,PK


In [10]:
# Add marker layer ontop of heat map
hotel_coordinates = list(zip(hotel_df["Lat"],hotel_df["Lon"]))

          

In [9]:
# Assign the marker layer to a variable
hover_text=[]

for hotel, city, country in zip (hotel_name, hotel_city_name, hotel_country):
    hover_text.append(f"{hotel},\n{city},\n{country}")
                      
markers = gmaps.marker_layer(hotel_coordinates, info_box_content = hover_text)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…